In [1]:
#increase the display size to 95% of window width
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))

In [12]:
import os
import shutil
import subprocess
import gc
import xarray as xr
import numpy as np
import pymp

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.transforms import Bbox
from matplotlib.patches import FancyBboxPatch
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

%matplotlib inline
path = os.getcwd().split('/')

machine_path = '/' + path[1] + '/' + path[2]

plt.style.use(machine_path+'/Doutorado/cenarios/mandyoc/scripts/des.mplstyle')

# Initial Setup

In [13]:
path = os.getcwd().split('/') # Get local file
machine_path = '/'+path[1]+'/'+path[2] # Select home according to OS.

# Path to the model directory
scenario = '/Doutorado/cenarios/mandyoc/keel/Lx3000km/keel_stable_DT200_HprodAst_Lx3000km'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx3000km/keel_stable_DT290_HprodAst_Lx3000km'
# scenario = '/Doutorado/cenarios/mandyoc/keel/Lx3000km/keel_stable_DT350_HprodAst_Lx3000km'

# local = True
local = False
if(local==True):
    model_path = machine_path + scenario
else:
    # scenario = '/RFT_Clc1_DT200_PT1292oC_lit80km_1x1km2_NHK'
    external_media = 'Joao_Macedo'
    model_path = f'/media/joao_macedo/{external_media}{scenario}'

read_ascii = True # if False, it will skip the ascii reading and saving processes and it will read the datasets from previous runs
save_images = True # Generate and save all the possible images
save_big_dataset = False#True
print(f'Model Path: {model_path}')

model_name = os.path.split(model_path)[1]
print(f'Model name: {model_name}')

# Create the output directory to save the dataset
output_path = os.path.join(model_path, "_output")
# print(output_path)

if not os.path.isdir(output_path):
    os.makedirs(output_path)

Model Path: /media/joao_macedo/Joao_Macedo/Doutorado/cenarios/mandyoc/keel/Lx3000km/keel_stable_DT200_HprodAst_Lx3000km
Model name: keel_stable_DT200_HprodAst_Lx3000km


# Reading Dataset

In [20]:
dataset = xr.open_dataset(f'{model_path}/_output_temperature.nc')

Nx = int(dataset.nx)
Nz = int(dataset.nz)
Lx = float(dataset.lx)
Lz = float(dataset.lz)

instant = dataset.time[-1]

x = np.linspace(0, Lx/1000.0, Nx)
z = np.linspace(Lz/1000.0, 0, Nz)
xx, zz  = np.meshgrid(x, z)

print(dataset.info)

<bound method Dataset.info of <xarray.Dataset>
Dimensions:      (time: 401, x: 301, z: 71)
Coordinates:
  * time         (time) float64 0.0 2.498 4.998 7.498 ... 995.0 997.5 1e+03
    step         (time) int32 ...
  * x            (x) float64 0.0 1e+04 2e+04 3e+04 ... 2.98e+06 2.99e+06 3e+06
  * z            (z) float64 -7e+05 -6.9e+05 -6.8e+05 ... -2e+04 -1e+04 0.0
Data variables:
    temperature  (time, x, z) float64 ...
Attributes: (12/94)
    nx:                                301
    nz:                                71
    lx:                                3000000.0
    lz:                                700000.0
    multigrid:                         1
    solver:                            direct
    ...                                ...
    temperature_units:                 C
    density_units:                     kg/m^3
    heat_units:                        W/m^3
    viscosity_units:                   Pa s
    strain_rate_units:                 s^(-1)
    pressure_units:

In [45]:
#selec regions
data = np.asarray(dataset.temperature[-1]).T

xleft = (xx>=200) & (xx <= 300)
xright = (xx>=1300) & (xx <= 1400)
xcenter = (xx>=700) & (xx <= 900) #craton
Datal = data[xleft].reshape(Nz, len(xleft[0][xleft[0]==True]))
Datar = data[xright].reshape(Nz, len(xright[0][xright[0]==True]))
Datac = data[xcenter].reshape(Nz, len(xcenter[0][xcenter[0]==True]))

# #calc horizontal mean
# Datal_mean = np.mean(Datal, axis=1)
# Datar_mean = np.mean(Datar, axis=1)
# Datac_mean = np.mean(Datac, axis=1)

In [44]:
d = np.asarray(dataset.temperature[-1]).T
# np.shape(d)
d[xleft].reshape(Nz, len(xleft[0][xleft[0]==True]))

array([[1700.63 , 1700.63 , 1700.63 , 1700.63 , 1700.63 , 1700.63 ,
        1700.63 , 1700.63 , 1700.63 , 1700.63 , 1700.63 ],
       [1656.   , 1656.94 , 1657.85 , 1658.74 , 1659.61 , 1660.45 ,
        1661.23 , 1661.82 , 1662.37 , 1663.14 , 1664.03 ],
       [1619.18 , 1620.59 , 1621.99 , 1623.37 , 1624.72 , 1626.05 ,
        1627.31 , 1628.44 , 1629.55 , 1630.8  , 1632.09 ],
       [1593.98 , 1595.36 , 1596.74 , 1598.12 , 1599.51 , 1600.89 ,
        1602.26 , 1603.63 , 1605.04 , 1606.42 , 1607.7  ],
       [1579.26 , 1580.28 , 1581.34 , 1582.43 , 1583.56 , 1584.72 ,
        1585.91 , 1587.16 , 1588.46 , 1589.7  , 1590.86 ],
       [1571.64 , 1572.22 , 1572.85 , 1573.54 , 1574.29 , 1575.09 ,
        1575.95 , 1576.86 , 1577.82 , 1578.79 , 1579.77 ],
       [1567.99 , 1568.15 , 1568.36 , 1568.64 , 1568.99 , 1569.39 ,
        1569.85 , 1570.38 , 1570.97 , 1571.62 , 1572.33 ],
       [1566.15 , 1565.96 , 1565.82 , 1565.74 , 1565.71 , 1565.74 ,
        1565.82 , 1565.98 , 1566.21 , 1566.